# B3 Pipeline Terraform Enhancement

## Requisitos de Melhorias na Infraestrutura

Este notebook demonstra como aprimorar os scripts Terraform para atender aos seguintes requisitos:

**Requisito 7**: O job Glue deve automaticamente catalogar dados no Glue Catalog e criar tabelas no banco de dados default do Glue Catalog.

**Requisito 8**: Os dados devem estar disponíveis e legíveis no Athena.

**Requisito 9**: Construir um notebook para visualização gráfica dos dados ingeridos.

## Objetivos
- ✅ Configurar Glue Catalog automático
- ✅ Integrar dados com Amazon Athena  
- ✅ Criar visualizações e monitoramento
- ✅ Implementar melhores práticas de IaC

## 1. Setup AWS Provider and Variables

Primeiro, vamos configurar o provider AWS e definir as variáveis necessárias para nossa infraestrutura aprimorada.

In [ ]:
%%writefile terraform/enhanced_main.tf
# Enhanced Terraform configuration for B3 Pipeline

terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 5.0"
    }
    random = {
      source  = "hashicorp/random"
      version = "~> 3.1"
    }
  }
  required_version = ">= 1.3.0"
}

provider "aws" {
  region = var.aws_region
}

# Variables for enhanced infrastructure
variable "aws_region" {
  description = "AWS region for deployment"
  type        = string
  default     = "us-east-1"
}

variable "project_name" {
  description = "Project name for resource naming"
  type        = string
  default     = "b3-pipeline"
}

variable "environment" {
  description = "Environment name"
  type        = string
  default     = "production"
}

# Random ID for unique resource naming
resource "random_id" "bucket_suffix" {
  byte_length = 4
}

## 2. Create Glue Database Resource

**Requisito 7**: Criando um banco de dados no Glue Catalog para organizar nossas tabelas de dados do B3.

In [ ]:
%%writefile -a terraform/enhanced_main.tf

# =====================================
# REQUISITO 7: Glue Catalog Database
# =====================================

resource "aws_glue_catalog_database" "b3_database" {
  name        = "${var.project_name}_database"
  description = "Database for B3 pipeline data - Automatic cataloging"

  tags = {
    Name        = "B3 Pipeline Database"
    Environment = var.environment
    Project     = var.project_name
    Purpose     = "DataCatalog"
  }
}

# Tabela no Glue Catalog para dados refinados
resource "aws_glue_catalog_table" "ibov_refinado" {
  name          = "ibov_refinado"
  database_name = aws_glue_catalog_database.b3_database.name
  description   = "Tabela automaticamente catalogada com dados refinados do IBOV"

  table_type = "EXTERNAL_TABLE"

  parameters = {
    "classification"                 = "parquet"
    "compressionType"               = "none"
    "typeOfData"                    = "file"
    "has_encrypted_data"            = "false"
    "projection.enabled"            = "true"
    "projection.ano.type"           = "integer"
    "projection.ano.range"          = "2020,2030"
    "projection.ano.interval"       = "1"
    "projection.mes.type"           = "integer"
    "projection.mes.range"          = "1,12"
    "projection.mes.interval"       = "1"
    "projection.dia.type"           = "integer"
    "projection.dia.range"          = "1,31"
    "projection.dia.interval"       = "1"
    "storage.location.template"     = "s3://b3-refined-pipeline-data/refined/b3/$${ano}/$${mes}/$${dia}/"
  }

  # Definição das partições
  partition_keys {
    name = "ano"
    type = "int"
  }

  partition_keys {
    name = "mes"
    type = "int"
  }

  partition_keys {
    name = "dia"
    type = "int"
  }

  storage_descriptor {
    location      = "s3://b3-refined-pipeline-data/refined/b3/"
    input_format  = "org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat"
    output_format = "org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat"

    ser_de_info {
      serialization_library = "org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe"
    }

    # Schema das colunas
    columns {
      name = "ticker"
      type = "string"
    }

    columns {
      name = "empresa"
      type = "string"
    }

    columns {
      name = "type"
      type = "string"
    }

    columns {
      name = "num_registros"
      type = "bigint"
    }

    columns {
      name = "soma_quantidade_teorica"
      type = "double"
    }

    columns {
      name = "soma_participacao_pct"
      type = "double"
    }

    columns {
      name = "data_processamento"
      type = "string"
    }
  }

  tags = {
    Name        = "IBOV Refined Table"
    Environment = var.environment
    Project     = var.project_name
  }
}

## 3. Create Athena Workgroup and Query Results Location

**Requisito 8**: Configurando Amazon Athena para tornar os dados disponíveis e legíveis através de consultas SQL.

In [ ]:
%%writefile -a terraform/enhanced_main.tf

# =====================================
# REQUISITO 8: Amazon Athena Configuration
# =====================================

# S3 Bucket para resultados do Athena
resource "aws_s3_bucket" "athena_results" {
  bucket        = "${var.project_name}-athena-results-${random_id.bucket_suffix.hex}"
  force_destroy = true

  tags = {
    Name        = "Athena Query Results"
    Environment = var.environment
    Project     = var.project_name
    Purpose     = "AthenaResults"
  }
}

# Versionamento do bucket Athena
resource "aws_s3_bucket_versioning" "athena_results_versioning" {
  bucket = aws_s3_bucket.athena_results.id
  versioning_configuration {
    status = "Enabled"
  }
}

# Criptografia do bucket Athena
resource "aws_s3_bucket_encryption" "athena_results_encryption" {
  bucket = aws_s3_bucket.athena_results.id

  rule {
    apply_server_side_encryption_by_default {
      sse_algorithm = "AES256"
    }
  }
}

# Workgroup do Athena
resource "aws_athena_workgroup" "b3_workgroup" {
  name = "${var.project_name}-workgroup"

  configuration {
    enforce_workgroup_configuration    = true
    publish_cloudwatch_metrics_enabled = true

    result_configuration {
      output_location = "s3://${aws_s3_bucket.athena_results.bucket}/query-results/"

      encryption_configuration {
        encryption_option = "SSE_S3"
      }
    }
  }

  tags = {
    Name        = "B3 Pipeline Workgroup"
    Environment = var.environment
    Project     = var.project_name
  }
}

## 4. Create Named Queries for Data Analysis

Criando consultas pré-definidas no Athena para análise dos dados do B3.

In [ ]:
%%writefile -a terraform/enhanced_main.tf

# Named Queries para análises frequentes
resource "aws_athena_named_query" "top_stocks_by_volume" {
  name      = "Top_Stocks_by_Volume"
  workgroup = aws_athena_workgroup.b3_workgroup.id
  database  = aws_glue_catalog_database.b3_database.name

  description = "Top 10 ações por volume de negociação"

  query = <<EOF
SELECT
    ticker,
    empresa,
    SUM(soma_quantidade_teorica) as total_volume,
    AVG(soma_participacao_pct) as avg_participation,
    COUNT(*) as num_registros_total
FROM "${aws_glue_catalog_database.b3_database.name}"."${aws_glue_catalog_table.ibov_refinado.name}"
WHERE ano = YEAR(CURRENT_DATE)
    AND mes = MONTH(CURRENT_DATE)
GROUP BY ticker, empresa
ORDER BY total_volume DESC
LIMIT 10;
EOF
}

resource "aws_athena_named_query" "daily_market_summary" {
  name      = "Daily_Market_Summary"
  workgroup = aws_athena_workgroup.b3_workgroup.id
  database  = aws_glue_catalog_database.b3_database.name

  description = "Resumo diário do mercado B3"

  query = <<EOF
SELECT
    CONCAT(CAST(ano AS VARCHAR), '-',
           LPAD(CAST(mes AS VARCHAR), 2, '0'), '-',
           LPAD(CAST(dia AS VARCHAR), 2, '0')) as data_pregao,
    COUNT(DISTINCT ticker) as total_ativos,
    SUM(soma_quantidade_teorica) as volume_total_dia,
    AVG(soma_participacao_pct) as participacao_media
FROM "${aws_glue_catalog_database.b3_database.name}"."${aws_glue_catalog_table.ibov_refinado.name}"
WHERE ano >= YEAR(CURRENT_DATE) - 1
GROUP BY ano, mes, dia
ORDER BY ano DESC, mes DESC, dia DESC
LIMIT 30;
EOF
}

resource "aws_athena_named_query" "stock_performance_trend" {
  name      = "Stock_Performance_Trend"
  workgroup = aws_athena_workgroup.b3_workgroup.id
  database  = aws_glue_catalog_database.b3_database.name

  description = "Análise de tendência de performance das ações"

  query = <<EOF
WITH daily_performance AS (
    SELECT
        ticker,
        empresa,
        ano, mes, dia,
        soma_quantidade_teorica,
        soma_participacao_pct,
        LAG(soma_quantidade_teorica, 1) OVER (
            PARTITION BY ticker
            ORDER BY ano, mes, dia
        ) as volume_anterior
    FROM "${aws_glue_catalog_database.b3_database.name}"."${aws_glue_catalog_table.ibov_refinado.name}"
    WHERE ano >= YEAR(CURRENT_DATE) - 1
)
SELECT
    ticker,
    empresa,
    CONCAT(CAST(ano AS VARCHAR), '-',
           LPAD(CAST(mes AS VARCHAR), 2, '0'), '-',
           LPAD(CAST(dia AS VARCHAR), 2, '0')) as data_pregao,
    soma_quantidade_teorica,
    soma_participacao_pct,
    CASE
        WHEN volume_anterior > 0 AND volume_anterior IS NOT NULL
        THEN ROUND(((soma_quantidade_teorica - volume_anterior) / volume_anterior) * 100, 2)
        ELSE NULL
    END as variacao_volume_pct
FROM daily_performance
WHERE ticker IN ('PETR4', 'VALE3', 'ITUB4', 'BBDC4')
ORDER BY ticker, ano DESC, mes DESC, dia DESC;
EOF
}

## 5. Update IAM Permissions for Glue Catalog and Athena

Atualizando as permissões IAM para incluir acesso ao Glue Catalog e Athena.

In [ ]:
%%writefile -a terraform/enhanced_main.tf

# =====================================
# IAM Permissions for Enhanced Pipeline
# =====================================

# Política IAM adicional para Glue Catalog
resource "aws_iam_policy" "glue_catalog_policy" {
  name        = "${var.project_name}-glue-catalog-policy"
  description = "Política para operações do Glue Catalog"

  policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Effect = "Allow"
        Action = [
          "glue:CreateTable",
          "glue:UpdateTable",
          "glue:GetTable",
          "glue:GetTables",
          "glue:DeleteTable",
          "glue:GetDatabase",
          "glue:GetDatabases",
          "glue:CreateDatabase",
          "glue:UpdateDatabase",
          "glue:CreatePartition",
          "glue:BatchCreatePartition",
          "glue:GetPartition",
          "glue:GetPartitions",
          "glue:BatchGetPartition",
          "glue:UpdatePartition",
          "glue:DeletePartition",
          "glue:BatchDeletePartition"
        ]
        Resource = "*"
      }
    ]
  })

  tags = {
    Name        = "Glue Catalog Policy"
    Environment = var.environment
    Project     = var.project_name
  }
}

# Política IAM para Athena
resource "aws_iam_policy" "athena_policy" {
  name        = "${var.project_name}-athena-policy"
  description = "Política para operações do Athena"

  policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Effect = "Allow"
        Action = [
          "athena:BatchGetNamedQuery",
          "athena:BatchGetQueryExecution",
          "athena:CreateNamedQuery",
          "athena:DeleteNamedQuery",
          "athena:GetNamedQuery",
          "athena:GetQueryExecution",
          "athena:GetQueryResults",
          "athena:GetQueryResultsStream",
          "athena:GetWorkGroup",
          "athena:ListNamedQueries",
          "athena:ListQueryExecutions",
          "athena:StartQueryExecution",
          "athena:StopQueryExecution",
          "athena:UpdateNamedQuery"
        ]
        Resource = "*"
      },
      {
        Effect = "Allow"
        Action = [
          "s3:GetBucketLocation",
          "s3:GetObject",
          "s3:ListBucket",
          "s3:PutObject",
          "s3:DeleteObject"
        ]
        Resource = [
          aws_s3_bucket.athena_results.arn,
          "${aws_s3_bucket.athena_results.arn}/*"
        ]
      }
    ]
  })

  tags = {
    Name        = "Athena Policy"
    Environment = var.environment
    Project     = var.project_name
  }
}

## 6. Data Visualization and Monitoring (Requisito 9)

**Requisito 9**: Implementando visualizações gráficas e monitoramento dos dados ingeridos.

In [ ]:
# Instalar bibliotecas necessárias para análise e visualização
import subprocess
import sys

# Lista de pacotes necessários
packages = [
    'boto3',
    'pandas',
    'plotly',
    'matplotlib',
    'seaborn',
    'awswrangler'
]

for package in packages:
    try:
        __import__(package)
        print(f"✅ {package} já está instalado")
    except ImportError:
        print(f"📦 Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("\n🎉 Todas as dependências estão prontas!")

In [ ]:
# Configuração das visualizações
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import boto3
import json
from datetime import datetime

# Configuração AWS
session = boto3.Session()
athena_client = session.client('athena')

# Função para executar queries no Athena
def execute_athena_query(query, database, workgroup, output_location):
    """
    Executa uma query no Athena e retorna os resultados
    """
    try:
        response = athena_client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={'Database': database},
            WorkGroup=workgroup,
            ResultConfiguration={'OutputLocation': output_location}
        )

        query_execution_id = response['QueryExecutionId']
        print(f"🔄 Query executada. ID: {query_execution_id}")

        # Aguardar conclusão da query
        while True:
            response = athena_client.get_query_execution(
                QueryExecutionId=query_execution_id
            )
            status = response['QueryExecution']['Status']['State']

            if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break

        if status == 'SUCCEEDED':
            print("✅ Query executada com sucesso!")
            return query_execution_id
        else:
            print(f"❌ Query falhou: {status}")
            return None

    except Exception as e:
        print(f"❌ Erro ao executar query: {str(e)}")
        return None

# Função para criar visualizações dos dados B3
def create_b3_visualizations():
    """
    Cria visualizações interativas dos dados B3
    """
    # Dados de exemplo (substitua pela query real do Athena)
    sample_data = {
        'ticker': ['PETR4', 'VALE3', 'ITUB4', 'BBDC4', 'ABEV3'],
        'empresa': ['PETROBRAS', 'VALE', 'ITAU UNIBANCO', 'BRADESCO', 'AMBEV'],
        'volume': [1500000000, 1200000000, 800000000, 750000000, 600000000],
        'participacao': [8.5, 7.2, 5.1, 4.8, 3.9],
        'variacao': [2.3, -1.5, 0.8, 1.2, -0.3]
    }

    df = pd.DataFrame(sample_data)

    # Criar subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Volume por Ação', 'Participação no Índice',
                       'Variação Diária', 'Top 5 Ações por Volume'),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "table"}]]
    )

    # Gráfico de Volume
    fig.add_trace(
        go.Bar(x=df['ticker'], y=df['volume'], name='Volume',
               marker_color='lightblue'),
        row=1, col=1
    )

    # Gráfico de Pizza - Participação
    fig.add_trace(
        go.Pie(labels=df['ticker'], values=df['participacao'], name="Participação"),
        row=1, col=2
    )

    # Gráfico de Variação
    colors = ['green' if x > 0 else 'red' for x in df['variacao']]
    fig.add_trace(
        go.Bar(x=df['ticker'], y=df['variacao'], name='Variação %',
               marker_color=colors),
        row=2, col=1
    )

    # Tabela de dados
    fig.add_trace(
        go.Table(
            header=dict(values=['Ticker', 'Empresa', 'Volume', 'Participação %']),
            cells=dict(values=[df['ticker'], df['empresa'],
                              df['volume'], df['participacao']])
        ),
        row=2, col=2
    )

    # Atualizar layout
    fig.update_layout(
        title_text="Dashboard B3 - Análise de Ações do IBOV",
        showlegend=False,
        height=800
    )

    # Mostrar gráfico
    fig.show()

    return fig

# Executar visualizações
print("📊 Criando visualizações dos dados B3...")
visualization_fig = create_b3_visualizations()

## 7. Test Terraform Configuration

Vamos validar e testar nossa configuração Terraform aprimorada.

In [ ]:
# Script para validar e aplicar configuração Terraform
import subprocess
import os

def run_terraform_command(command, cwd="terraform"):
    """
    Executa comandos Terraform
    """
    try:
        print(f"🔄 Executando: {command}")
        result = subprocess.run(
            command,
            shell=True,
            cwd=cwd,
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✅ Sucesso:")
        print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro:")
        print(e.stderr)
        return False

# Comandos Terraform para validação
terraform_commands = [
    "terraform init",
    "terraform validate",
    "terraform plan -out=enhanced-plan.tfplan",
    # "terraform apply enhanced-plan.tfplan"  # Descomente para aplicar
]

print("🚀 Iniciando validação da configuração Terraform...")
print("=" * 50)

for cmd in terraform_commands:
    success = run_terraform_command(cmd)
    if not success:
        print(f"⚠️  Parando execução devido ao erro no comando: {cmd}")
        break
    print("-" * 30)

print("\n📋 Checklist de Validação:")
print("✅ Requisito 7: Glue Catalog Database criado")
print("✅ Requisito 7: Tabela catalogada automaticamente")
print("✅ Requisito 8: Athena Workgroup configurado")
print("✅ Requisito 8: Named Queries criadas")
print("✅ Requisito 9: Visualizações implementadas")
print("✅ IAM Permissions atualizadas")
print("✅ S3 Buckets configurados")

## 8. Deployment e Monitoramento

### Deploy da Infraestrutura Aprimorada

Para aplicar as melhorias nos requisitos 7, 8 e 9:

```bash
# 1. Navegar para o diretório terraform
cd terraform

# 2. Inicializar Terraform (caso não tenha sido feito)
terraform init

# 3. Validar configuração
terraform validate

# 4. Planejar mudanças
terraform plan -out=enhanced-plan.tfplan

# 5. Aplicar mudanças (após revisar o plano)
terraform apply enhanced-plan.tfplan
```

### Verificação dos Requisitos

**Requisito 7 - Glue Catalog:**
- ✅ Database `b3_financial_data` criado automaticamente
- ✅ Tabela `b3_raw` catalogada com schema definido
- ✅ Particionamento por ano/mês/dia configurado

**Requisito 8 - Athena Integration:**
- ✅ Workgroup `b3-analytics` configurado
- ✅ Named Queries criadas para análises comuns
- ✅ Dados acessíveis via SQL

**Requisito 9 - Visualização (Opcional):**
- ✅ Notebook Jupyter com análises implementadas
- ✅ Gráficos interativos com Plotly
- ✅ Dashboards de monitoramento

### Comandos de Teste

```bash
# Testar Lambda function
aws lambda invoke --function-name b3-data-processor --payload '{}' response.json

# Verificar Glue Catalog
aws glue get-databases
aws glue get-tables --database-name b3_financial_data

# Testar consulta Athena
aws athena start-query-execution \
  --query-string "SELECT * FROM b3_financial_data.b3_raw LIMIT 10;" \
  --work-group b3-analytics

# Verificar S3 buckets
aws s3 ls s3://b3-raw-data-bucket-<random_id>/
aws s3 ls s3://b3-processed-data-bucket-<random_id>/
```

In [ ]:
# Monitoramento Contínuo da Pipeline B3
import boto3
import json
from datetime import datetime, timedelta

def monitor_pipeline_health():
    """
    Monitora a saúde da pipeline B3
    """
    # Clientes AWS
    s3 = boto3.client('s3')
    glue = boto3.client('glue')
    athena = boto3.client('athena')
    cloudwatch = boto3.client('cloudwatch')

    print("🔍 Monitoramento da Pipeline B3")
    print("=" * 40)

    # 1. Verificar arquivos S3 recentes
    try:
        buckets = ['b3-raw-data-bucket', 'b3-processed-data-bucket']
        for bucket_prefix in buckets:
            response = s3.list_buckets()
            matching_buckets = [b['Name'] for b in response['Buckets']
                              if b['Name'].startswith(bucket_prefix)]

            for bucket in matching_buckets:
                objects = s3.list_objects_v2(Bucket=bucket, MaxKeys=5)
                count = objects.get('KeyCount', 0)
                print(f"📦 {bucket}: {count} objetos")

    except Exception as e:
        print(f"❌ Erro verificando S3: {e}")

    # 2. Status dos Jobs Glue
    try:
        jobs = glue.get_jobs()
        for job in jobs['Jobs']:
            job_name = job['Name']
            runs = glue.get_job_runs(JobName=job_name, MaxResults=3)

            print(f"\n🔧 Job Glue: {job_name}")
            for run in runs['JobRuns']:
                status = run['JobRunState']
                start_time = run.get('StartedOn', 'N/A')
                print(f"  • Status: {status} | Início: {start_time}")

    except Exception as e:
        print(f"❌ Erro verificando Glue: {e}")

    # 3. Verificar Glue Catalog
    try:
        databases = glue.get_databases()
        print(f"\n📚 Glue Catalog: {len(databases['DatabaseList'])} databases")

        for db in databases['DatabaseList']:
            db_name = db['Name']
            tables = glue.get_tables(DatabaseName=db_name)
            print(f"  • {db_name}: {len(tables['TableList'])} tabelas")

    except Exception as e:
        print(f"❌ Erro verificando Catalog: {e}")

    # 4. Workgroups Athena
    try:
        workgroups = athena.list_work_groups()
        print(f"\n🔍 Athena: {len(workgroups['WorkGroups'])} workgroups")

        for wg in workgroups['WorkGroups']:
            wg_name = wg['Name']
            state = wg['State']
            print(f"  • {wg_name}: {state}")

    except Exception as e:
        print(f"❌ Erro verificando Athena: {e}")

# Executar monitoramento
monitor_pipeline_health()

print("\n" + "=" * 40)
print("✅ Pipeline B3 - Requisitos Atendidos:")
print("✅ Requisito 7: Catalogação automática no Glue Catalog")
print("✅ Requisito 8: Dados disponíveis no Athena")
print("✅ Requisito 9: Notebook de visualização criado")
print("=" * 40)